In [3]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pandas as pd

In [130]:
conf = SparkConf()
conf.setMaster("local[*]").setAppName("into_group")

In [132]:

sc = SparkContext.getOrCreate(conf)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCo

In [131]:
sc.stop()

In [7]:
spark = SQLContext(sc)

23/02/14 10:53:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/home/saputra/belajar/data-batch-processing/belajar_batch/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# GROUP Section

In [54]:
df_green = spark.read.option("header",True).parquet("./data/pq/green/*/*")

In [9]:
df_green.registerTempTable("green")

/home/saputra/belajar/data-batch-processing/belajar_batch/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [56]:
df_green = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 

    -- Revenue sum
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
-- ORDER BY 
--     1, 2
""")

In [11]:
df_green.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
| 260|2020-01-01 02:00:00|235.16999999999996|            20|
| 260|2020-01-01 07:00:00|             67.25|             5|
|  25|2020-01-02 15:00:00| 749.4300000000001|            29|
|  83|2020-01-05 22:00:00|             59.66|             5|
|  41|2020-01-16 10:00:00| 549.0500000000001|            42|
| 116|2020-01-13 09:00:00|            583.07|            29|
|  97|2020-01-17 17:00:00| 783.5600000000002|            50|
|  74|2020-01-07 03:00:00|               7.8|             1|
|   3|2020-01-06 12:00:00|            122.49|             6|
|  82|2020-01-11 01:00:00|            501.49|            42|
| 254|2020-01-05 18:00:00|43.120000000000005|             2|
|  65|2020-01-28 16:00:00| 702.1600000000001|            28|
|  38|2020-01-31 11:00:00|            105.58|             3|
|  33|2020-01-25 16:00:0

In [12]:
df_green.write.parquet("./data/report/revenue/green",mode="overwrite")

23/02/14 10:53:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/14 10:53:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 10:53:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 10:53:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 10:53:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/02/14 10:53:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 10:53:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 10:53:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 10:53:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [57]:

df_yellow = spark.read.option("header",True).parquet("./data/pq/yellow/*/*")

In [14]:
# spark.dropTempTable("yellowNew")
# df_yellow = spark.sql("""
#     DROP VIEW IF EXISTS yellowNew
# """)

In [58]:
df_yellow.registerTempTable("yellow")

/home/saputra/belajar/data-batch-processing/belajar_batch/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [59]:
df_yellow = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 

    -- Revenue sum
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
-- ORDER BY 
--     1, 2
""")

In [25]:
df_yellow.repartition(20).write.parquet("./data/report/revenue/yellow",mode="overwrite")

23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 10:56:15 WARN MemoryManager: Total allocation exceeds 95.

# JOIN Section

In [44]:
df_yellow = spark.read.option("header",True).parquet("./data/report/revenue/yellow/*")

In [32]:
df_yellow.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|  48|2021-01-07 16:00:00|1330.5100000000002|            85|
| 201|2021-02-03 10:00:00|             67.09|             1|
| 129|2020-11-28 18:00:00|             67.15|             3|
|  76|2021-01-18 14:00:00|              83.2|             3|
|  86|2020-08-26 04:00:00|              36.0|             1|
|  25|2021-05-17 15:00:00|             21.72|             2|
| 234|2021-03-19 17:00:00|2820.4300000000003|           158|
| 243|2020-10-20 08:00:00|              12.3|             1|
| 113|2020-06-26 15:00:00|219.46000000000004|            15|
| 211|2020-10-20 00:00:00|             63.75|             4|
| 132|2020-02-16 08:00:00| 8785.680000000004|           181|
| 186|2020-02-03 01:00:00|1341.3799999999997|            69|
|  13|2020-07-07 09:00:00|             92.22|             5|
| 133|2020-10-02 12:00:0

In [43]:
df_green = spark.read.option("header",True).parquet("./data/report/revenue/green/")

In [34]:
df_green.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
| 226|2020-03-07 09:00:00|             73.33|             5|
|  75|2020-01-28 19:00:00|1299.0000000000002|            97|
|  65|2020-01-30 00:00:00|             75.37|             6|
| 181|2020-01-25 20:00:00|            366.33|            26|
| 129|2020-01-26 04:00:00| 497.1500000000001|            39|
|  25|2020-01-13 18:00:00|            215.32|            16|
|  80|2020-01-22 11:00:00|             57.18|             4|
|  65|2020-01-07 20:00:00|            349.12|            22|
|  41|2020-01-09 09:00:00| 955.1400000000002|            69|
| 129|2020-01-15 10:00:00|221.47000000000003|            12|
| 135|2020-01-08 08:00:00|            112.88|             4|
| 228|2020-01-15 18:00:00|             35.91|             2|
| 213|2020-01-08 07:00:00|            342.44|            12|
| 244|2020-01-17 11:00:0

In [68]:
df_green = df_green.withColumnRenamed("amount","green_amount").withColumnRenamed("number_records","green_number_records")

In [69]:
df_yellow = df_yellow.withColumnRenamed("amount","yellow_amount").withColumnRenamed("number_records","yellow_number_records")

In [60]:
df_join = df_green.join(df_yellow,["zone","hour"],how="outer")

In [52]:
df_join.write.parquet("./data/report/revenue/total")

23/02/14 11:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/14 11:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 11:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 11:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 11:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/02/14 11:07:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 11:07:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 11:07:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 11:07:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [114]:
df_join = spark.read.option("header",True).parquet("./data/report/revenue/total/")

In [108]:
df_join.show()

+----+-------------------+------------+--------------------+------------------+---------------------+
|zone|               hour|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+----+-------------------+------------+--------------------+------------------+---------------------+
|   1|2020-01-01 09:00:00|        null|                null|            175.26|                    2|
|   1|2020-01-02 14:00:00|        null|                null|            628.52|                    6|
|   1|2020-01-02 21:00:00|        null|                null|             100.3|                    1|
|   1|2020-01-04 05:00:00|        null|                null|              45.8|                    1|
|   1|2020-01-06 16:00:00|        null|                null|             165.6|                    2|
|   1|2020-01-07 15:00:00|        null|                null|            117.96|                    1|
|   1|2020-01-08 02:00:00|        null|                null|             24.36|   

# Outer Join

In [78]:
df_join.filter(df_join["green_amount"].isNotNull()).show()

+----+-------------------+------------------+--------------------+-------------+---------------------+
|zone|               hour|      green_amount|green_number_records|yellow_amount|yellow_number_records|
+----+-------------------+------------------+--------------------+-------------+---------------------+
|   1|2020-02-06 07:00:00|              98.3|                   1|        82.61|                    2|
|   1|2020-03-21 17:00:00|             100.3|                   1|         null|                 null|
|   1|2020-10-15 18:00:00|            126.79|                   1|         null|                 null|
|   1|2020-11-10 08:00:00|             101.3|                   1|         null|                 null|
|   1|2021-04-12 07:00:00|              92.8|                   1|         null|                 null|
|   3|2020-01-02 18:00:00|              16.8|                   1|         null|                 null|
|   3|2020-01-03 11:00:00|25.470000000000002|                   2|       

In [79]:
df_join.filter(df_join["yellow_amount"].isNotNull()).show()

+----+-------------------+------------+--------------------+------------------+---------------------+
|zone|               hour|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+----+-------------------+------------+--------------------+------------------+---------------------+
|   1|2020-01-01 09:00:00|        null|                null|            175.26|                    2|
|   1|2020-01-02 14:00:00|        null|                null|            628.52|                    6|
|   1|2020-01-02 21:00:00|        null|                null|             100.3|                    1|
|   1|2020-01-04 05:00:00|        null|                null|              45.8|                    1|
|   1|2020-01-06 16:00:00|        null|                null|             165.6|                    2|
|   1|2020-01-07 15:00:00|        null|                null|            117.96|                    1|
|   1|2020-01-08 02:00:00|        null|                null|             24.36|   

In [80]:
df_join.filter((df_join["yellow_amount"].isNotNull() & df_join["green_amount"].isNotNull())).show()

+----+-------------------+------------+--------------------+------------------+---------------------+
|zone|               hour|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+----+-------------------+------------+--------------------+------------------+---------------------+
|   1|2020-02-06 07:00:00|        98.3|                   1|             82.61|                    2|
|   3|2020-01-16 07:00:00|       45.24|                   1|             37.74|                    1|
|   3|2020-01-27 08:00:00|       55.04|                   3|             33.85|                    1|
|   3|2020-01-28 07:00:00|       45.24|                   1|              29.0|                    1|
|   3|2020-01-31 15:00:00|       30.48|                   1|              40.8|                    1|
|   3|2020-02-09 16:00:00|      163.82|                   1|             35.28|                    1|
|   3|2020-02-10 11:00:00|       53.22|                   1|              36.0|   

In [115]:
df_zone = spark.read.option("header",True).parquet("./data/pq/zone/")

In [116]:
df_result = df_join.join(df_zone,df_zone.LocationID == df_join.zone)

In [117]:
df_result.show()

+----+-------------------+------------+--------------------+------------------+---------------------+----------+-------+--------------+------------+
|zone|               hour|green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|Borough|     zone_name|service_zone|
+----+-------------------+------------+--------------------+------------------+---------------------+----------+-------+--------------+------------+
|   1|2020-01-01 09:00:00|        null|                null|            175.26|                    2|         1|    EWR|Newark Airport|         EWR|
|   1|2020-01-02 14:00:00|        null|                null|            628.52|                    6|         1|    EWR|Newark Airport|         EWR|
|   1|2020-01-02 21:00:00|        null|                null|             100.3|                    1|         1|    EWR|Newark Airport|         EWR|
|   1|2020-01-04 05:00:00|        null|                null|              45.8|                    1|     

In [118]:
df_result.drop(df_result.LocationID).write.parquet("./data/temp/revenue-zones")

23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/14 12:03:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/14 12:03:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
